In [1]:
#imports necessary modules
import cv2
import matplotlib
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
import numpy as np
import os,glob
import math
#%matplotlib inline 
import plotutils
import metadata
exiftoolPath = None
if os.name == 'nt':
    exiftoolPath = 'C:\\Windows\\exiftool.exe'
import utils as msutils
import pandas as pd
import sys
import PIL
from PIL import Image
from PIL.ExifTags import TAGS, GPSTAGS
import piexif
import pyexiv2
import tempfile

In [9]:
#imports modules for plotting with colour 

def plotwithcolorbar(img, title='', show=True):
    ''' Plot an image with a colorbar '''
    fig, axis = plt.subplots(1, 1, figsize=(8, 6))
    rad2 = axis.imshow(img)
    axis.set_title(title)
    divider = make_axes_locatable(axis)
    cax = divider.append_axes("right", size="3%", pad=0.05)
    fig.colorbar(rad2, cax=cax)
    fig1=plt.gcf()
    plt.tight_layout()
    if show == True:
        plt.show()
    return fig1

def colormap(cmap):
    ''' Set the defalut plotting colormap
    Could be one of 'gray, viridis, plasma, inferno, magma, nipy_spectral'
    '''
    plt.set_cmap(cmap)

In [10]:
#calls the calibration images we want to use for correcting the photos, one from the beginning and one from the end of the flight 
def callcalibrationimages(calibrationimage, imagePath):
    imageName = os.path.join(imagePath, calibrationimage)
    imageRaw = plt.imread(imageName)
    colormap('viridis');
    fig = plotwithcolorbar(imageRaw, title='Raw image values with colorbar')

In [11]:
#radiance to reflectance for calibration panel images, chose one from the beginning of the flight and one from the end 
def calibrating (calibrationimage1, calibrationimage2, imagePath,ulx1,uly1,lrx1,lry1,ulx2,uly2,lrx2,lry2,plots=True):
    
    ##calibrating first image 
    imageName1 = os.path.join(imagePath, calibrationimage1)
    imageRaw1 = plt.imread(imageName1)
    meta = metadata.Metadata(imageName1, exiftoolPath=exiftoolPath)
    cameraMake = meta.get_item('EXIF:Make')
    cameraModel = meta.get_item('EXIF:Model')
    firmwareVersion = meta.get_item('EXIF:Software')
    bandName = meta.get_item('XMP:BandName')
    radianceImage1, L, V, R = msutils.raw_image_to_radiance(meta, imageRaw1)
    markedImg1 = radianceImage1.copy()
    ulx1 = ulx1
    uly1 = uly1
    lrx1 = lrx1
    lry1 = lry1
    cv2.rectangle(markedImg1,(ulx1,uly1),(lrx1,lry1),(0,255,0),3)
    #specific for each calibration image 
    panelCalibration1 = { 
    "Blue": 0.49, 
    "Green": 0.49, 
    "Red": 0.49, 
    "Red edge": 0.49, 
    "NIR": 0.49
    }
    #may have to change coordinate order so values read from low to high, otherwise will get a Nan return 
    panelRegion1 = radianceImage1[uly1:lry1, lrx1:ulx1]
    meanRadiance1 = panelRegion1.mean()
    panelReflectance1 = panelCalibration1[bandName]
    radianceToReflectance1 = panelReflectance1 / meanRadiance1
    if plots == True:
        plotutils.plotwithcolorbar(markedImg1, 'Panel region in radiance image')
        print('Mean Radiance in panel region: {:1.3f} W/m^2/nm/sr'.format(meanRadiance1))
        print('Radiance to reflectance conversion factor: {:1.3f}'.format(radianceToReflectance1))
        reflectanceImage1 = radianceImage1 * radianceToReflectance1
        plotutils.plotwithcolorbar(reflectanceImage1, 'Converted Reflectane Image')
        
    ##calibrating the second image     
    imageName2 = os.path.join(imagePath, calibrationimage2)
    imageRaw2 = plt.imread(imageName2)
    meta = metadata.Metadata(imageName2, exiftoolPath=exiftoolPath)
    cameraMake = meta.get_item('EXIF:Make')
    cameraModel = meta.get_item('EXIF:Model')
    firmwareVersion = meta.get_item('EXIF:Software')
    bandName = meta.get_item('XMP:BandName')
    radianceImage2, L, V, R = msutils.raw_image_to_radiance(meta, imageRaw2)
    markedImg2 = radianceImage2.copy()
    ulx2 = ulx2
    uly2 = uly2
    lrx2 = lrx2
    lry2 = lry2
    cv2.rectangle(markedImg2,(ulx2,uly2),(lrx2,lry2),(0,255,0),3)
    #specific to each calibration panel 
    panelCalibration2 = { 
    "Blue": 0.49, 
    "Green": 0.49, 
    "Red": 0.49, 
    "Red edge": 0.49, 
    "NIR": 0.49 
    }
    #may have to change coordinate order so values read from low to high, else will get a Nan return 
    panelRegion2 = radianceImage2[uly2:lry2, lrx2:ulx2]
    meanRadiance2 = panelRegion2.mean()
    panelReflectance2 = panelCalibration2[bandName]
    radianceToReflectance2 = panelReflectance2 / meanRadiance2
    if plots == True:
        plotutils.plotwithcolorbar(markedImg2, 'Panel region in radiance image')
        print('Mean Radiance in panel region: {:1.3f} W/m^2/nm/sr'.format(meanRadiance2))
        print('Radiance to reflectance conversion factor: {:1.3f}'.format(radianceToReflectance2))
        reflectanceImage2 = radianceImage2 * radianceToReflectance2
        plotutils.plotwithcolorbar(reflectanceImage2, 'Converted Reflectane Image')
        
    ##get the mean radiance to reflectance between the panel images    
    meanradiancetoreflectance = ((radianceToReflectance1 + radianceToReflectance2)/2)
    return meanradiancetoreflectance


In [12]:
##check the reflectance of your panel photo, consistent reflectance with a low standard deviation is desired in the panel region
def calibrationcheck (calibrationimage, imagePath, ulx, uly, lrx, lry):
    imageName = os.path.join(imagePath, calibrationimage)
    imageRaw = plt.imread(imageName)
    meta = metadata.Metadata(imageName, exiftoolPath=exiftoolPath)
    bandName = meta.get_item('XMP:BandName')
    ulx = ulx
    uly = uly
    lrx = lrx
    lry = lry
    panelCalibration = { 
    "Blue": 0.49, 
    "Green": 0.49, 
    "Red": 0.49, 
    "Red edge": 0.49, 
    "NIR": 0.49 
    }
    radianceImage, L, V, R = msutils.raw_image_to_radiance(meta, imageRaw)
    panelRegion = radianceImage[uly:lry, lrx:ulx]
    meanRadiance = panelRegion.mean()
    panelReflectance = panelCalibration[bandName]
    radianceToReflectance = panelReflectance / meanRadiance
    reflectanceImage = radianceImage * radianceToReflectance
    panelRegionRaw = imageRaw[uly:lry, lrx:ulx]
    panelRegionRefl = reflectanceImage[uly:lry, lrx:ulx]
    print('Min Reflectance in panel region: {:1.2f}'.format(panelRegionRefl.min()))
    print('Max Reflectance in panel region: {:1.2f}'.format(panelRegionRefl.max()))
    print('Mean Reflectance in panel region: {:1.2f}'.format(panelRegionRefl.mean()))
    print('Standard deviation in region: {:1.4f}'.format(panelRegionRefl.std()))

In [13]:
##transfers metadata from one image to another 
def copyMeta(inputIMGpath,outputIMGpath):
    original_meta = pyexiv2.ImageMetadata(inputIMGpath)
    original_meta.read()
    new_meta = pyexiv2.ImageMetadata(outputIMGpath)
    new_meta.read()
    original_meta.copy(new_meta,comment=False)
    new_meta.write()

In [15]:
##function for calibrating flight images based on the panel photos, also corrects for distortion 
def calibration(imagefile,calibrationimage1, calibrationimage2,calibrationPath,imagePath,figpath,figname,ulx1,uly1,lrx1,lry1,ulx2,uly2,lrx2,lry2,raw=False, show=False):
    exiftoolPath = 'C:\\Windows\\exiftool.exe'
    meta = metadata.Metadata(os.path.join(imagePath,imagefile), exiftoolPath=exiftoolPath)
    flightImageName = os.path.join(imagePath, imagefile)
    flightImageRaw=plt.imread(flightImageName)
    if raw == True: #lets you show or not show the original 
        plotutils.plotwithcolorbar(flightImageRaw, 'Raw Image')
    flightRadianceImage, _, _, _ = msutils.raw_image_to_radiance(meta, flightImageRaw)
    radianceToReflectance = calibrating(calibrationimage1, calibrationimage2, calibrationPath,ulx1,uly1,lrx1,lry1,ulx2,uly2,lrx2,lry2,plots=False) #might need to adjust for two images
    flightReflectanceImage = flightRadianceImage * radianceToReflectance
    flightUndistortedReflectance = msutils.correct_lens_distortion(meta, flightReflectanceImage)
    a = plotwithcolorbar(flightUndistortedReflectance, 'Reflectance converted and undistorted image', show=False) #pop up the images in jupyter or just save to dorectory 
    a.savefig(os.path.join(figpath,figname))
    copyMeta(os.path.join(imagePath,imagefile),os.path.join(figpath,figname))
    return 

In [16]:
##example for how to use 
#imagePath = os.path.join('C:\\', 'Users', 'Sara', 'Pictures', 'MICASENSE', '0006SET', '000', '2nd Flight') #where the images are 
#calibrationImage1 = 'IMG_0079_4.tif' 
#calibrationImage2 = 'IMG_0338_5.tif'
#calibrationPath = os.path.join('C:\\', 'Users', 'Sara', 'Pictures', 'MICASENSE', '0006SET', '000','panel images', 'USE THESE')
#figpath = os.path.join('C:\\', 'Users', 'Sara', 'Pictures', 'MICASENSE', '0006SET', '000', 'correction graph') #where the correction should save to 
#files = os.listdir(os.path.join('C:\\', 'Users', 'Sara', 'Pictures', 'MICASENSE', '0006SET', '000', '2nd FLight')) #the directory of images 
#for imagefile in files:
    #if imagefile.endswith(".tif"): #to only take the tif's 
        #figname = imagefile[:-4] + '_corrected.tif' #renaming for correction 
        #calibration(imagefile,calibrationImage1, calibrationImage2, calibrationPath, imagePath,figpath,figname,1150,200,980,350,810,500,630,700, raw=False, show=False)